In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import sys
import copy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
import pandas as pd
import pickle

torch.manual_seed(42)

# 하이퍼파라미터 설정
batch_size = 32
epochs = 10
lr = 0.01
PCA_dim = 8
CLS_num = 2



with open('./data/data.pkl','rb') as file:
    data = pickle.load(file)
X = data['X']
y = data['Y']



x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=70)



def Fit_to_quantum(X,PCA_dim):
    pca = PCA(n_components=PCA_dim)
    X_pca = pca.fit_transform(X)
    return X_pca




# PyTorch Tensor로 변환
x_train_pca, y_train = torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
x_test_pca, y_test = torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)


class Feature_data_loader(Dataset):
    def __init__(self,x_train,y_train):
        self.feature1 = x_train
        temp = copy.deepcopy(x_train)
        shuffle = torch.randperm(len(temp))
        self.feature2 = temp[shuffle]
        self.y1 = y_train
        temp_y = copy.deepcopy(y_train)
        self.y2 = temp_y[shuffle]

    def __len__(self):
        return len(self.feature1)
    def __getitem__(self,idx):
        input1 = self.feature1[idx]
        input2 = self.feature2[idx]
        if self.y1[idx] == self.y2[idx]:
            label = torch.tensor(1.).float()
        else:
            label = torch.tensor(0.).float()
        return [input1,input2],label


# DataLoader 생성


feature_loader = DataLoader(Feature_data_loader(x_train_pca, y_train.float()),batch_size=batch_size,shuffle=True)
test_feature_loader = DataLoader(Feature_data_loader(x_test_pca, y_test.float()),batch_size=len(x_test_pca),shuffle=False)
train_loader = DataLoader(TensorDataset(x_train_pca, y_train.float()), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test_pca, y_test.float()), batch_size=len(x_test_pca), shuffle=False)


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from pennylane import numpy as np
from collections import OrderedDict
import math
from functions.training import Early_stop_train


result_list_classical = []
# Pennylane 장치 설정
dev = qml.device("default.qubit", wires=PCA_dim)


def ZZFeatureMapLayer(features, wires):
    """사용자 정의 ZZFeatureMap 레이어"""
    index = 0
    for i in wires:
        qml.Hadamard(wires=i)
        qml.RZ(features[:,index], wires=i)
        index += 1

    for j in range(0, len(wires)-1):
        qml.CNOT(wires=[j, j+1])
        qml.RZ((features[:,index]), wires=j+1)
        qml.CNOT(wires=[j, j+1])
        index+=1

def ZZFeatureMapLayer_fixed(features, wires):
    """사용자 정의 ZZFeatureMap 레이어"""
    index = 0
    for i in wires:
        qml.Hadamard(wires=i)
        qml.RZ(features[:,index], wires=i)
        index += 1
    index=0
    for j in range(0, len(wires)-1):
        qml.CNOT(wires=[j, j+1])
        qml.RZ((features[:,index])*(features[:,index+1]), wires=j+1)
        qml.CNOT(wires=[j, j+1])
        index+=1

def ansatz(params):
    for j in range(len(params)):
        # 각 큐비트에 대해 RX, RY, RZ 회전 적용
        for i in range(len(params[0])):
            qml.RY(params[j, i, 0], wires=i)
            qml.RZ(params[j, i, 1], wires=i)

        # 인접한 큐비트 간 CNOT 게이트로 엔탱글링
        if j == len(params)-1:
            pass
        else:
            for i in range(len(params[0])-1):
                qml.CNOT(wires=[i, i+1])


# 양자 레이어 정의
@qml.qnode(dev, interface='torch', diff_method="backprop")
def QuantumLayer(features,params):
    ZZFeatureMapLayer(features, wires=range(PCA_dim))
    ansatz(params)
    return qml.probs(wires=range(math.ceil(math.log2(CLS_num))))


## 양자 커널
@qml.qnode(dev, interface='torch', diff_method="backprop")
def Kernal(features1,features2):
    ZZFeatureMapLayer(features1, wires=range(PCA_dim))
    qml.adjoint(ZZFeatureMapLayer)(features2,wires=range(PCA_dim))
    return qml.probs(wires=range(PCA_dim))




class Feature_model(nn.Module):
    def __init__(self):
        super(Feature_model,self).__init__()
        self.cls = nn.Sequential(OrderedDict([('cls1', nn.Linear(PCA_dim,PCA_dim*8)),
                                              ('relu1', nn.ReLU()),('cls2', nn.Linear(PCA_dim*8,PCA_dim*8)),
                                              ('relu4', nn.ReLU()),('cls5', nn.Linear(PCA_dim*8,PCA_dim*2-1)),
                                             ]))
        self.Kernal = Kernal
    def forward(self,inputs):
        epsilon = 1e-6
        input1 = inputs[0]
        input2 = inputs[1]
        input1 = self.cls(input1)
        input2 = self.cls(input2)
        output = self.Kernal(input1,input2)
        output = output.type(torch.float32)
        return output[:,0].clamp(min=epsilon, max=1-epsilon)





# 하이브리드 모델 정의
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        @qml.qnode(dev, interface='torch', diff_method="backprop")
        def QuantumLayer(features,params):
            for i in range(1):
                ZZFeatureMapLayer(features, wires=range(PCA_dim))
                ansatz(params[i])
            return qml.probs(wires=range(math.ceil(math.log2(CLS_num))))
        self.cls = feature_model.cls
        self.quantum_layer = QuantumLayer
        self.Q_params = nn.Parameter((torch.rand([3,PCA_dim,PCA_dim,2])*2-1)*np.pi,requires_grad=True)
    def forward(self, x):
        epsilon = 1e-6
        x = self.cls(x)
        #print(qml.draw(self.quantum_layer)(x,self.Q_params))
        # 양자 레이어 정의
        
        quantum_output = self.quantum_layer(x,self.Q_params)
        quantum_output = quantum_output.type(torch.float32)
        return quantum_output[:,0].clamp(min=epsilon, max=1-epsilon)

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.cls_layer_1 = nn.Linear(PCA_dim,PCA_dim*PCA_dim)
        self.cls_layer_2 = nn.Linear(PCA_dim*PCA_dim,PCA_dim*PCA_dim-1)
        self.output_layer = nn.Linear(PCA_dim*PCA_dim-1,PCA_dim)
    def forward(self, x):
        x = self.cls_layer_1(x)
        x = nn.ReLU()(x)
        x = self.cls_layer_2(x)
        x = nn.ReLU()(x)
        output = self.output_layer(x)
        return output
# 모델, 손실 함수, 최적화 설정


feature_model = Feature_model(); criterion = nn.BCELoss()
#model = Model(); criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(feature_model.parameters(), lr=0.01)


# 모델 학습 및 평가
train_process = Early_stop_train(feature_model, optimizer, criterion)
train_process.train_model(feature_loader,test_feature_loader,epochs=50,res=15)
 
model = HybridModel(); criterion = nn.BCELoss()
for param in model.cls.parameters():
    param.requires_grad = False
#model.load_state_dict(para_dict)
optimizer = optim.Adam(model.parameters(), lr=0.01)
print('\n\n Test start \n\n')
train_process = Early_stop_train(model, optimizer, criterion)
train_process.train_model(train_loader,test_loader,epochs=50,res=5)

_,acc = train_process.test(test_loader)
result_list_classical.append(acc)
print(f"Test Accuracy: {acc:.2f}")




Test set: Average loss: 0.7700, Accuracy: 36.000001430511475/60 (60%)
Epoch 1 Loss 0.635768 acc : 0.654167 stop count : 0
Test set: Average loss: 0.6408, Accuracy: 42.999998331069946/60 (72%)
Epoch 2 Loss 0.315522 acc : 0.887500 stop count : 0
Test set: Average loss: 0.4995, Accuracy: 47.000001668930054/60 (78%)
Epoch 3 Loss 0.188824 acc : 0.950000 stop count : 0
Test set: Average loss: 0.4950, Accuracy: 47.000001668930054/60 (78%)
Epoch 4 Loss 0.095911 acc : 0.981250 stop count : 0
Test set: Average loss: 0.3681, Accuracy: 48.99999976158142/60 (82%)
Epoch 5 Loss 0.076939 acc : 0.987500 stop count : 0
Test set: Average loss: 0.4112, Accuracy: 48.00000071525574/60 (80%)
Epoch 6 Loss 0.046054 acc : 1.000000 stop count : 1
Test set: Average loss: 0.4485, Accuracy: 48.00000071525574/60 (80%)
Epoch 7 Loss 0.031431 acc : 1.000000 stop count : 2
Test set: Average loss: 0.3770, Accuracy: 49.999998807907104/60 (83%)
Epoch 8 Loss 0.023117 acc : 1.000000 stop count : 3
Test set: Average loss: 0.

KeyboardInterrupt: 

In [ ]:
# 하이브리드 모델 정의
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        @qml.qnode(dev, interface='torch', diff_method="backprop")
        def QuantumLayer(features,params):
            for i in range(5):
                ZZFeatureMapLayer(features, wires=range(PCA_dim))
                ansatz(params[i])
            return qml.probs(wires=range(math.ceil(math.log2(CLS_num))))
        self.cls = feature_model.cls
        self.quantum_layer = QuantumLayer
        self.Q_params = nn.Parameter((torch.rand([5,PCA_dim,PCA_dim,2])*2-1)*np.pi,requires_grad=True)
    def forward(self, x):
        epsilon = 1e-6
        x = self.cls(x)
        #print(qml.draw(self.quantum_layer)(x,self.Q_params))
        # 양자 레이어 정의
        
        quantum_output = self.quantum_layer(x,self.Q_params)
        quantum_output = quantum_output.type(torch.float32)
        return quantum_output[:,0].clamp(min=epsilon, max=1-epsilon)
model = HybridModel(); criterion = nn.BCELoss()
"""
for param in model.cls.parameters():
    param.requires_grad = False"""
#model.load_state_dict(para_dict)
optimizer = optim.Adam(model.parameters(), lr=0.01)
print('\n\n Test start \n\n')
train_process = Early_stop_train(model, optimizer, criterion)
train_process.train_model(train_loader,test_loader,epochs=50,res=5)

_,acc = train_process.test(test_loader)
result_list_classical.append(acc)
print(f"Test Accuracy: {acc:.2f}")




 Test start 



Test set: Average loss: 0.6997, Accuracy: 62.000001311302185/133 (47%)
Epoch 1 Loss 0.618027 acc : 0.721577 stop count : 0
Test set: Average loss: 0.5795, Accuracy: 103.0000034570694/133 (77%)
Epoch 2 Loss 0.544142 acc : 0.759077 stop count : 0
Test set: Average loss: 0.5533, Accuracy: 103.0000034570694/133 (77%)
Epoch 3 Loss 0.519517 acc : 0.766964 stop count : 0
Test set: Average loss: 0.5443, Accuracy: 99.99999904632568/133 (75%)
Epoch 4 Loss 0.504613 acc : 0.795089 stop count : 0
Test set: Average loss: 0.5411, Accuracy: 103.99999964237213/133 (78%)
Epoch 5 Loss 0.494703 acc : 0.799851 stop count : 0
Test set: Average loss: 0.5371, Accuracy: 103.99999964237213/133 (78%)
Epoch 6 Loss 0.497868 acc : 0.801042 stop count : 0
Test set: Average loss: 0.5311, Accuracy: 103.99999964237213/133 (78%)
Epoch 7 Loss 0.486640 acc : 0.801488 stop count : 0
Test set: Average loss: 0.5273, Accuracy: 101.99999934434891/133 (77%)
Epoch 8 Loss 0.487218 acc : 0.802827 stop count : 0
T

In [ ]:
for x,y in feature_loader:
    temp1 = feature_model.cls(x[0])
    temp2 = feature_model.cls(x[1])
    dist = torch.sum(torch.cos((abs((temp1[0][:8])-(temp2[0][:8])))))
    res = feature_model(x)
    print(f"x0 : {temp1[0]}\nx1 : {temp2[0]}\nTrue : {y[0]}\npred : {res[0]}\ndist : {dist} \n")

x0 : tensor([-0.3032, -0.2752,  0.2603,  0.4664, -0.2640, -0.1456, -0.2868,  0.8981,
        -0.3989,  0.5444,  0.1930,  0.2267, -0.4411, -0.3035,  0.0068])
x1 : tensor([ 1.1239, -0.0359, -1.3022, -0.2710,  0.5643, -0.3913, -0.2144, -0.7197,
         0.1363, -0.1955,  1.0148,  1.1243,  0.0656, -0.7225,  0.0306])
True : 0.0
pred : 0.05115753412246704
dist : 4.459741115570068 

x0 : tensor([ 0.4485, -0.0349, -0.5096, -0.3922,  0.0418, -0.7581, -0.1731, -0.6040,
        -0.6219,  0.2528,  1.0591,  0.8221, -0.3526,  0.0514, -0.0806])
x1 : tensor([ 0.8221, -0.0677, -0.9179, -0.3599,  0.2982, -0.9064, -0.3512, -0.7823,
        -0.3727,  0.1729,  1.2668,  1.1445, -0.2365, -0.2614, -0.1920])
True : 1.0
pred : 0.8180967569351196
dist : 7.7723894119262695 

x0 : tensor([ 0.2734, -0.2145, -0.7478, -1.0233, -0.1127, -0.8802, -0.3109, -1.1384,
        -1.2338,  0.1295,  1.1245,  0.7705, -0.3301,  0.6847, -0.0226])
x1 : tensor([ 0.6313, -0.0151, -0.6760, -0.5029,  0.1141, -0.5093, -0.0304, -0.5661,


In [ ]:
model = HybridModel(); criterion = nn.BCELoss()
for param in model.cls.parameters():
    param.requires_grad = False
#model.load_state_dict(para_dict)
optimizer = optim.Adam(model.parameters(), lr=0.01)
print('\n\n Test start \n\n')
train_process = Early_stop_train(model, optimizer, criterion)
train_process.train_model(train_loader,test_loader,epochs=50,res=5)

_,acc = train_process.test(test_loader)
result_list_classical.append(acc)
print(f"Test Accuracy: {acc:.2f}")




 Test start 



Test set: Average loss: 3.2300, Accuracy: 257.99999952316284/300 (86%)
Epoch 1 Loss 0.532657 acc : 0.978693 stop count : 0
Test set: Average loss: 2.4358, Accuracy: 265.9999988079071/300 (89%)
Epoch 2 Loss 0.379985 acc : 0.981534 stop count : 0
Test set: Average loss: 2.0270, Accuracy: 263.9999988079071/300 (88%)
Epoch 3 Loss 0.284708 acc : 0.977273 stop count : 0
Test set: Average loss: 1.7702, Accuracy: 264.9999988079071/300 (88%)
Epoch 4 Loss 0.225733 acc : 0.977178 stop count : 0
Test set: Average loss: 1.6297, Accuracy: 263.9999988079071/300 (88%)
Epoch 5 Loss 0.188910 acc : 0.977273 stop count : 0
Test set: Average loss: 1.5429, Accuracy: 264.9999988079071/300 (88%)
Epoch 6 Loss 0.162743 acc : 0.977273 stop count : 0
Test set: Average loss: 1.4786, Accuracy: 263.9999988079071/300 (88%)
Epoch 7 Loss 0.138512 acc : 0.980469 stop count : 0

KeyboardInterrupt: 

In [ ]:
from functions.training import Kernal_method
kernal = Kernal_method(feature_model)
kernal.train(torch.tensor(x_train).float(),torch.tensor(y_train),torch.tensor(x_test).float(),torch.tensor(y_test))

C:\Users\pop75\AppData\Local\Temp\ipykernel_6284\3766126110.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  kernal.train(torch.tensor(x_train).float(),torch.tensor(y_train),torch.tensor(x_test).float(),torch.tensor(y_test))
100%|██████████| 140/140 [00:15<00:00,  8.80it/s]
c:\Users\pop75\OneDrive\Desktop\Project\AI_quantum_project\Quantum_KAN_modif\functions\training.py:221: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(y_train).float()
c:\Users\pop75\OneDrive\Desktop\Project\AI_quantum_project\Quantum_KAN_modif\functions\training.py:237: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().

 acc :  0.9166666865348816


(tensor([[1.0000e+00, 4.3082e-04, 1.0700e-04,  ..., 9.1076e-01, 4.2808e-03,
          9.9550e-01],
         [4.3082e-04, 1.0000e+00, 9.8370e-01,  ..., 1.5583e-03, 8.7080e-01,
          2.6220e-04],
         [1.0700e-04, 9.8370e-01, 1.0000e+00,  ..., 6.1900e-03, 9.4014e-01,
          1.7317e-04],
         ...,
         [9.1076e-01, 1.5583e-03, 6.1900e-03,  ..., 1.0000e+00, 3.2334e-02,
          9.3087e-01],
         [4.2808e-03, 8.7080e-01, 9.4014e-01,  ..., 3.2334e-02, 1.0000e+00,
          5.9778e-03],
         [9.9550e-01, 2.6220e-04, 1.7317e-04,  ..., 9.3087e-01, 5.9778e-03,
          1.0000e+00]]),
 tensor([[1.9169e-03, 9.1796e-01, 9.7190e-01,  ..., 2.1166e-02, 9.8791e-01,
          2.9666e-03],
         [9.8851e-01, 6.6410e-04, 1.5055e-04,  ..., 8.8462e-01, 2.9582e-03,
          9.8935e-01],
         [3.2924e-02, 6.5025e-01, 7.5476e-01,  ..., 1.1498e-01, 9.2022e-01,
          4.0211e-02],
         ...,
         [6.8599e-04, 9.5061e-01, 9.9008e-01,  ..., 1.3445e-02, 9.7771e-01,
   